In [ ]:
import sys; sys.path.append('../../')

In [ ]:
from jpdr.datasets import GroZi3k

In [ ]:
ds_gallery = GroZi3k('../../data/GroceryProducts_Tonioni/', subset='val_gallery')
ds_query = GroZi3k('../../data/GroceryProducts_Tonioni/', subset='val_query')

In [ ]:
df_gal = ds_gallery.df
df_query = ds_query.df

In [ ]:
import numpy as np

prod_ids = np.concatenate(df_query['target'].apply(lambda t: t['product_ids']).values)
uniq_prod_ids = np.unique(prod_ids)
print(f'{len(uniq_prod_ids)} product ids')

In [ ]:
print(f'{len(df_query)} rek-afbeeldingen')

In [ ]:
from PIL import Image, ImageDraw

im, target = ds_query[7]

# draw = ImageDraw.Draw(im)

# for box in target['boxes']:
#     draw.rectangle(tuple(box), width=5)
w, h = im.size
scale = 0.25
im.resize((int(w*scale), int(h*scale)))

In [ ]:
from tqdm.auto import tqdm

for i, box in enumerate(tqdm(target['boxes'])):
    crop = im.crop([int(x) for x in box])
    w, h = crop.size
    crop.resize((int(w*scale), int(h*scale))).save(f'box_{i}.jpg')

In [ ]:
gal_idxs = ds_gallery.df[ds_gallery.df['product_id'].isin(target['product_ids'].tolist())].index

for idx in gal_idxs:
    im, label = ds_gallery[idx]
    w, h = im.size
    im.resize((int(w*scale), int(h*scale))).save(f'product_{label}.jpg')

In [ ]:
from jpdr.utils.crop_batch import crop_and_batch_boxes
from torchvision.transforms.functional import to_pil_image, to_tensor
from jpdr.transforms import RandomShortestSize
import jpdr.utils.boxes as B

resize = RandomShortestSize(min_size=800)
im, target = ds_query[7]
im, target = resize(im, target)
im = to_tensor(im)

crop_size = 500

crop_boxes = B.make_boxes_batchable(
    target['boxes'], to_int=True, crop_box_size=crop_size,
)
#crop_boxes = [B.drop_boxes_outside_img(im, img_boxes) for img_boxes in crop_boxes]
#crop_boxes = [B.drop_overlapping_boxes(img_boxes, 0.5) for img_boxes in crop_boxes]
crops = crop_and_batch_boxes(im[None, :], crop_boxes[None, :])
crops = [to_pil_image(crop) for crop in crops]

for i, crop in enumerate(crops):
    crop.save(f'crop_{crop_size}_{i}.jpg')